In [1]:
import gzip
from collections import defaultdict
import math
import scipy.optimize
import numpy
import string
import random
from sklearn import linear_model
import numpy as np

In [2]:
import homework3

In [3]:
def readGz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)

In [4]:
def readCSV(path):
    f = gzip.open(path, 'rt')
    f.readline()
    for l in f:
        u,b,r = l.strip().split(',')
        r = int(r)
        yield u,b,r

In [5]:
def countRight(a,b,epsilon):
    if len(a) != len(b):
        print("It looks like your solution has the wrong length (got " + str(len(a)) + ", expected "
 + str(len(b)) + ")")
        return 0
    a_ = np.array(a).flatten()
    b_ = np.array(b).flatten()
    right = np.abs(a_ - b_) < epsilon
    return float(sum(right) / len(right))

In [6]:
# Some data structures that will be useful

In [7]:
allRatings = []
for l in readCSV("../Datasets/assignment1/train_Interactions.csv.gz"):
    allRatings.append(l)

In [8]:
len(allRatings)

200000

In [9]:
allRatings[0]

('u67805239', 'b61372131', 4)

In [10]:
ratingsTrain = allRatings[:190000]
ratingsValid = allRatings[190000:]
ratingsPerUser = defaultdict(list)
ratingsPerItem = defaultdict(list)
for u,b,r in ratingsTrain:
    ratingsPerUser[u].append((b,r))
    ratingsPerItem[b].append((u,r))

In [11]:
##################################################
# Rating prediction                              #
##################################################

In [12]:
trainRatings = [r[2] for r in ratingsTrain]
globalAverage = homework3.getGlobalAverage(trainRatings)

In [13]:
def testQ1():
    ga = homework3.getGlobalAverage(trainRatings)
    trivialValidMSE = homework3.trivialValidMSE(ratingsValid, ga)

    print("average = " + str(ga))
    print("validation MSE = " + str(trivialValidMSE))

In [14]:
testQ1()

average = 3.6868052631578947
validation MSE = 1.6802113179223874


In [15]:
def iterateN(which, alpha, betaU, betaI, lamb, N):
    for i in range(N):
        alpha = which.alphaUpdate(ratingsTrain, alpha, betaU, betaI, lamb)
        betaU = which.betaUUpdate(ratingsPerUser, alpha, betaU, betaI, lamb)
        betaI = which.betaIUpdate(ratingsPerItem, alpha, betaU, betaI, lamb)
        mse, mseReg = which.msePlusReg(ratingsTrain, alpha, betaU, betaI, lamb)
        print("Iteration " + str(i + 1))
        print("  MSE = " + str(mse))
        print("  regularized objective = " + str(mseReg))
    return alpha, betaU, betaI, mse, mseReg

In [16]:
def testModel(which):
    betaU = {}
    betaI = {}
    for u in ratingsPerUser:
        betaU[u] = 0

    for b in ratingsPerItem:
        betaI[b] = 0

    alpha = globalAverage # Could initialize anywhere, this is a guess

    alpha, betaU, betaI, mse, mseReg = iterateN(which, alpha, betaU, betaI, 1.0, 1)
    validMSE = which.validMSE(ratingsValid, alpha, betaU, betaI)

    return alpha, betaU, betaI, mse, mseReg, validMSE

In [17]:
def testQ2():
    alpha, betaU, betaI, mse, mseReg, validMSE = testModel(homework3)
    print("validMSE = " + str(validMSE))

In [18]:
testQ2()

Iteration 1
  MSE = 1.0727154704481001
  regularized objective = 13249.828429085357
validMSE = 1.440670105511256


In [19]:
testQ2()

Iteration 1
  MSE = 1.0727154704481001
  regularized objective = 13249.828429085357
validMSE = 1.440670105511256


In [20]:
def testQ3():
    betaU = {}
    betaI = {}
    for u in ratingsPerUser:
        betaU[u] = 0

    for b in ratingsPerItem:
        betaI[b] = 0

    alpha = globalAverage # Could initialize anywhere, this is a guess

    alpha, betaU, betaI = homework3.goodModel(ratingsTrain, ratingsPerUser, ratingsPerItem, alpha, betaU, betaI)
    validMSE = homework3.validMSE(ratingsValid, alpha, betaU, betaI)

    print("validMSE = " + str(validMSE))

In [21]:
testQ3()

validMSE = 1.4345367814517573


In [22]:
##################################################
# Read prediction                                #
##################################################

In [23]:
# From baseline code
bookCount = defaultdict(int)
totalRead = 0

for user,book,_ in readCSV("../Datasets/assignment1/train_Interactions.csv.gz"):
    bookCount[book] += 1
    totalRead += 1

mostPopular = [(bookCount[x], x) for x in bookCount]
mostPopular.sort()
mostPopular.reverse()

In [24]:
def testQ4():
    readValid, notRead = homework3.generateValidation(allRatings, ratingsValid)
    print("Should be equal: " + str((len(readValid), len(notRead), len(ratingsValid))))

In [25]:
testQ4()

Should be equal: (10000, 9999, 10000)


In [26]:
def testQ5():
    return1 = homework3.baseLineStrategy(mostPopular, totalRead)
    better = homework3.improvedStrategy(mostPopular, totalRead)

    readValid, notRead = homework3.generateValidation(allRatings, ratingsValid)

    correctA = homework3.evaluateStrategy(return1, readValid, notRead)
    correctB = homework3.evaluateStrategy(better, readValid, notRead)

    print("Accuracy (simple strategy) = " + str(correctA))
    print("Accuracy (better strategy) = " + str(correctB))

In [27]:
testQ5()

Accuracy (simple strategy) = 0.70975
Accuracy (better strategy) = 0.7495


In [28]:
def testQ6():
    readValid, notRead = homework3.generateValidation(allRatings, ratingsValid)

    for (u,b) in list(readValid)[:20] + list(notRead)[:20]:
        a = homework3.jaccardThresh(u,b,ratingsPerItem,ratingsPerUser)
        print("Jaccard-based predictor for " + str((u,b)) + " = " + str(a))

    # This is slow (so the autograder doesn't run it) but you should run it at home once you have a good solution
    #homework3.writePredictionsRead(ratingsPerItem, ratingsPerUser)

In [29]:
testQ6()

Jaccard-based predictor for ('u07431008', 'b16024414') = 1
Jaccard-based predictor for ('u12960922', 'b30494145') = 1
Jaccard-based predictor for ('u78843013', 'b81323559') = 1
Jaccard-based predictor for ('u46058032', 'b15644590') = 0
Jaccard-based predictor for ('u66628649', 'b91708984') = 0
Jaccard-based predictor for ('u58489030', 'b80469048') = 0
Jaccard-based predictor for ('u88475939', 'b87439329') = 1
Jaccard-based predictor for ('u43951342', 'b90422949') = 0
Jaccard-based predictor for ('u43794298', 'b50869133') = 1
Jaccard-based predictor for ('u10115672', 'b79928088') = 1
Jaccard-based predictor for ('u91609906', 'b32921266') = 0
Jaccard-based predictor for ('u88903696', 'b56757068') = 0
Jaccard-based predictor for ('u28385354', 'b08171212') = 0
Jaccard-based predictor for ('u48112936', 'b77333717') = 1
Jaccard-based predictor for ('u35776343', 'b56779366') = 1
Jaccard-based predictor for ('u49177254', 'b62670070') = 1
Jaccard-based predictor for ('u86383480', 'b46584434') =

In [30]:
##################################################
# Category prediction                            #
##################################################

In [31]:
data = []

for d in readGz("../Datasets/assignment1/train_Category.json.gz"):
    data.append(d)
    # Just use a little data to make things faster...
    if len(data) > 10000:
        break

In [32]:
wordCount = defaultdict(int)
punctuation = set(string.punctuation)
for d in data:
    r = ''.join([c for c in d['review_text'].lower() if not c in punctuation])
    for w in r.split():
        wordCount[w] += 1

counts = [(wordCount[w], w) for w in wordCount]
counts.sort()
counts.reverse()

In [33]:
NW = 500 # dictionary size

In [34]:
words = [x[1] for x in counts[:NW]]

In [35]:
wordId = dict(zip(words, range(len(words))))
wordSet = set(words)

In [36]:
def testQ7():
    f1 = homework3.featureCat(data[0], words, wordId, wordSet)

    print("Feature vector = " + str(f1))

In [37]:
testQ7()

Feature vector = [0, 2, 2, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [38]:
def testQ8():
    X = [homework3.featureCat(d, words, wordId, wordSet) for d in data]
    y = [d['genreID'] for d in data]

    Xtrain = X[:9*len(X)//10]
    ytrain = y[:9*len(y)//10]
    Xvalid = X[9*len(X)//10:]
    yvalid = y[9*len(y)//10:]

    mod = linear_model.LogisticRegression(C=1)
    mod.fit(Xtrain, ytrain)
    pred = mod.predict(Xvalid)
    correctA = pred == yvalid
    correctA = sum(correctA) / len(correctA)

    X = homework3.betterFeatures(data)
    Xtrain = X[:9*len(X)//10]
    Xvalid = X[9*len(X)//10:]

    mod = linear_model.LogisticRegression(C=1)
    mod.fit(Xtrain, ytrain)
    pred = mod.predict(Xvalid)
    correctB = pred == yvalid
    correctB = sum(correctB) / len(correctB)

    sc = correctA < (correctB * 0.99)

    data_test = []
    for d in readGz("../Datasets/assignment1/test_Category.json.gz"):
        data_test.append(d)

    Xtest = homework3.betterFeatures(data_test)
    pred_test = mod.predict(Xtest)

    homework3.writePredictionsCategory(pred_test)

    if sc:
        print("Looks like your solution is better")
    else:
        print("Looks like your solution is not better")

In [ ]:
testQ8()